In [1]:
from langchain.vectorstores import FAISS
from langchain.embeddings import GPT4AllEmbeddings
from typing import List, Dict, Tuple

vector_db_path = "vectorstores/db_faiss"

# Initialize the embedding model
embedding_model = GPT4AllEmbeddings(model_file="models/all-MiniLM-L6-v2-f16.gguf")

# Load the FAISS vector database with the embedding model
db = FAISS.load_local(folder_path=vector_db_path, embeddings=embedding_model, allow_dangerous_deserialization = True)

# Perform a similarity search
def search_query(query: str, k: int = 30):
    """
    Perform a similarity search on the vector database.
    
    :param query: The query string.
    :param k: The number of top results to return.
    :return: The top results as a list of strings.
    """
    results = db.similarity_search(query, k=k)
    return [result.page_content for result in results]

# Example query
query = "Hồ sơ đề nghị cấp chứng chỉ hành nghề dược đối với người nước ngoài gồm những nội dung nào"
top_results = search_query(query, k=30)

# Print results
print("\nTop Results:")
for result in top_results:
    print(result)



Top Results:
Chương II CHỨNG CHỈ HÀNH NGHỀ DƯỢC > Mục 1. HỒ SƠ, THỦ TỤC CẤP, CẤP LẠI, ĐIỀU CHỈNH NỘI DUNG VÀ THU HỒI CHỨNG CHỈ HÀNH NGHỀ DƯỢC > Điều 3. Quy định chi tiết về hồ sơ đề nghị cấp Chứng chỉ hành nghề dược > 2. Đối với các giấy tờ do cơ quan có thẩm quyền nước ngoài cấp phải được hợp pháp hóa lãnh sự theo quy định. Các giấy tờ này phải có bản dịch sang tiếng Việt và được công chứng theo quy định.: 
Chương II CHỨNG CHỈ HÀNH NGHỀ DƯỢC > Mục 1. HỒ SƠ, THỦ TỤC CẤP, CẤP LẠI, ĐIỀU CHỈNH NỘI DUNG VÀ THU HỒI CHỨNG CHỈ HÀNH NGHỀ DƯỢC > Điều 6. Quy định chi tiết về thủ tục cấp, cấp lại, điều chỉnh nội dung Chứng chỉ hành nghề dược > 12. Trường hợp Chứng chỉ hành nghề dược được cấp lại theo quy định tại khoản 8 Điều 24 của Luật dược, người đề nghị cấp lại Chứng chỉ hành nghề dược không phải nộp phí.: 
Chương II CHỨNG CHỈ HÀNH NGHỀ DƯỢC > Mục 1. HỒ SƠ, THỦ TỤC CẤP, CẤP LẠI, ĐIỀU CHỈNH NỘI DUNG VÀ THU HỒI CHỨNG CHỈ HÀNH NGHỀ DƯỢC > Điều 6. Quy định chi tiết về thủ tục cấp, cấp lại, điều 

In [ ]:
query = "Thủ tục cấp chứng chỉ hành nghề dược gồm các nội dung gì?"
top_results = search_query(query, k=30)

# Print results
print("\nTop Results:")
for result in top_results:
    print(result)



Top Results:
Chương II CHỨNG CHỈ HÀNH NGHỀ DƯỢC > Mục 1. HỒ SƠ, THỦ TỤC CẤP, CẤP LẠI, ĐIỀU CHỈNH NỘI DUNG VÀ THU HỒI CHỨNG CHỈ HÀNH NGHỀ DƯỢC > Điều 6. Quy định chi tiết về thủ tục cấp, cấp lại, điều chỉnh nội dung Chứng chỉ hành nghề dược > 12. Trường hợp Chứng chỉ hành nghề dược được cấp lại theo quy định tại khoản 8 Điều 24 của Luật dược, người đề nghị cấp lại Chứng chỉ hành nghề dược không phải nộp phí.: 
Chương II CHỨNG CHỈ HÀNH NGHỀ DƯỢC > Mục 1. HỒ SƠ, THỦ TỤC CẤP, CẤP LẠI, ĐIỀU CHỈNH NỘI DUNG VÀ THU HỒI CHỨNG CHỈ HÀNH NGHỀ DƯỢC > Điều 6. Quy định chi tiết về thủ tục cấp, cấp lại, điều chỉnh nội dung Chứng chỉ hành nghề dược > 8. Chứng chỉ hành nghề dược được lập thành 02 (hai) bản: 01 (một) bản cấp cho cá nhân đề nghị cấp, cấp lại, điều chỉnh nội dung Chứng chỉ hành nghề dược; 01 bản lưu tại cơ quan cấp Chứng chỉ hành nghề dược.: 
Chương II CHỨNG CHỈ HÀNH NGHỀ DƯỢC > Mục 1. HỒ SƠ, THỦ TỤC CẤP, CẤP LẠI, ĐIỀU CHỈNH NỘI DUNG VÀ THU HỒI CHỨNG CHỈ HÀNH NGHỀ DƯỢC > Điều 6. Quy định 

: 

In [ ]:
from transformers import AutoTokenizer, pipeline
from ipex_llm.transformers import AutoModelForCausalLM
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_community.vectorstores import FAISS
from transformers import BitsAndBytesConfig

import torch

# Define the path for the model
model_file = "Qwen/Qwen2-1.5B-Instruct"
# model_file = "vilm/vinallama-7b-chat"

# Define the FAISS vector store path
vector_db_path = "vectorstores/db_faiss"

# Load the Hugging Face model and tokenizer
def load_huggingface_model(model_file):
    # quantization_config = BitsAndBytesConfig(
    #     load_in_4bit=True,  # Tải trọng số được lượng hóa trước theo định dạng 4 bit
    #     bnb_4bit_quant_type="nf4",  # Sử dụng loại lượng hóa "nf4" cho trọng số 4 bit
    #     bnb_4bit_compute_dtype=torch.bfloat16,  # Sử dụng torch.bfloat16 cho các phép tính trung gian
    #     bnb_4bit_use_double_quant=True,  # Sử dụng độ chính xác kép để lượng hóa kích hoạt
    # )
    # quantization_config = BitsAndBytesConfig(load_in_8bit=True, llm_int8_threshold = 6.0)
    model = AutoModelForCausalLM.from_pretrained(model_file, cpu_embedding=True, trust_remote_code=True, load_in_4bit = True)
    model = model.to('xpu')
    tokenizer = AutoTokenizer.from_pretrained(model_file, trust_remote_code=True)
    print('Successfully loaded Tokenizer and optimized Model!')
    return model, tokenizer

# Read the vector database (FAISS)
def read_vectors_db():
    embedding_model = GPT4AllEmbeddings(model_file="models/all-MiniLM-L6-v2-f16.gguf")
    db = FAISS.load_local(vector_db_path, embedding_model, allow_dangerous_deserialization=True)
    return db

# Perform similarity search on the vector database
def search_vector_db(query, k=2):
    db = read_vectors_db()
    results = db.similarity_search(query, k=k)
    return [result.page_content for result in results]
# Load the model and tokenizer
model, tokenizer = load_huggingface_model(model_file)


system_prompt = "Bạn là một trợ lí Tiếng Việt nhiệt tình và trung thực. Hãy luôn trả lời một cách hữu ích nhất có thể."
template = '''Chú ý các yêu cầu sau:
- Câu trả lời phải chính xác và đầy đủ nếu ngữ cảnh có câu trả lời. 
- Chỉ sử dụng các thông tin có trong ngữ cảnh được cung cấp.
- Chỉ cần từ chối trả lời và không suy luận gì thêm nếu ngữ cảnh không có câu trả lời.
Hãy trả lời câu hỏi dựa trên ngữ cảnh:
### Ngữ cảnh :
{context}

### Câu hỏi :
{question}

### Trả lời :'''





c:\Users\84866\miniforge3\envs\LLM\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
question = '''Hồ sơ đề nghị cấp chứng chỉ hành nghề  dược đối với người nước ngoài gồm những nội dung nào, trả lời một cách chi tiết và đầy đủ nhất'''
context_list = search_vector_db(question, k = 50)
context = "\n".join(context_list)
conversation = [{"role": "system", "content": system_prompt }]
conversation.append({"role": "user", "content": template.format(context = context, question = question)})

with torch.inference_mode():
    text = tokenizer.apply_chat_template(
        conversation,
        tokenize=False,
        add_generation_prompt=True)
    model_inputs = tokenizer(text,return_tensors="pt").to(model.device)
    attention_mask = model_inputs["attention_mask"]
    _ = model.generate(model_inputs.input_ids,
                        do_sample=False,
                        max_new_tokens=32) # warm-up
    generated_ids = model.generate(
        model_inputs.input_ids,
        attention_mask=attention_mask,
        max_new_tokens=2048,
        temperature = 0.5,
        #top_p=0.95,
        #top_k=40,
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [ ]:
import textwrap

# Example text (you can replace this with your query result)
text = response

# Function to wrap text
def wrap_text(text, width=80):
    wrapper = textwrap.TextWrapper(width=width, replace_whitespace=False)
    wrapped_text = "\n".join([wrapper.fill(line) for line in text.splitlines()])
    return wrapped_text                                                                                                                                                                                            

# Wrap and print the text
wrapped_text = wrap_text(text, width=140)
print(wrapped_text)


Dựa trên ngữ cảnh được cung cấp, hồ sơ đề nghị cấp Chứng chỉ hành nghề dược đối với người nước ngoài bao gồm các nội dung sau:

1. **Giấy tờ do cơ quan có thẩm quyền nước ngoài cấp**:
   - Các giấy tờ này phải được hợp pháp hóa lãnh sự theo quy định.
   - Các giấy tờ này phải có bản dịch sang tiếng Việt và được công chứng theo quy định.

2. **Số lượng hồ sơ**:
   - Hồ sơ đề nghị cấp Chứng chỉ hành nghề dược thực hiện theo quy định tại Điều 24 của Luật dược và được quy định cụ thể như sau:
     - **Điều 3, Khoản 3**: Số lượng hồ sơ quy định tại Điều này là 01 bộ.

3. **Thủ tục nộp hồ sơ**:
   - Người đề nghị cấp Chứng chỉ hành nghề dược nộp hồ sơ trực tiếp hoặc gửi qua đường bưu điện về cơ quan có thẩm quyền.

4. **Thời hạn xử lý**:
   - Trong thời hạn 05 ngày làm việc, kể từ ngày cấp, cấp lại, điều chỉnh nội dung Chứng chỉ hành nghề dược, cơ quan tiếp nhận hồ sơ công
bố, cập nhật trên Cổng thông tin điện tử của đơn vị.

5. **Trách nhiệm của cơ quan cấp Chứng chỉ**:
   - Cơ quan cấp Chứ

In [1]:
import os
os.environ["SYSL_CACHE_PERSISTENT"] = "1"
os.environ["BIGDL_LLM_XMX_DISABLED"] = "1"

In [2]:
# from langchain_community.embeddings import IpexLLMBgeEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(
    model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    model_kwargs = {"device": "cpu"},
    encode_kwargs = {"normalize_embeddings": True}
)

C:\Users\84866\AppData\Local\Temp\ipykernel_15884\716098532.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(
c:\Users\84866\miniforge3\envs\llm\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\84866\miniforge3\envs\llm\Lib\site-packages\huggingface_hub\file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. I

In [3]:
from icecream import ic
query = "Quy định chi tiết về hồ sơ đề nghị cấp Chứng chỉ hành nghề dược?"
text_embeddings = embedding_model.embed_query(query)
print(text_embeddings)

[0.017037129029631615, 0.08424554020166397, -0.047518301755189896, -0.019289856776595116, -0.007699376437813044, 0.062378499656915665, -0.035556331276893616, 0.04276996850967407, -0.006690417416393757, 0.018267089501023293, -0.016419829800724983, -0.00724531477317214, -0.07636758685112, -0.017232734709978104, -0.08785039931535721, -0.05165267735719681, 0.03056853450834751, -0.02021319419145584, 0.061916500329971313, -0.006997369695454836, 0.09413452446460724, 0.04276726394891739, 0.05157408118247986, -0.07482123374938965, 0.006027231924235821, 0.03691814839839935, -0.061101898550987244, -0.022436179220676422, 0.00867445021867752, -0.062334660440683365, -0.043436646461486816, 0.09009597450494766, -0.010157124139368534, -0.009579943493008614, -0.01028712559491396, -0.057528797537088394, -0.0008064216235652566, 0.02838105708360672, 0.05306721851229668, 0.04574000835418701, 0.03575751930475235, -0.07261887937784195, -0.055187441408634186, 0.10263773053884506, -0.026521896943449974, 0.02674